In [2]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

import torch
from torch.utils.data import Dataset, DataLoader 

from datasets import Dataset
import evaluate
from transformers import DefaultDataCollator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

2022-11-22 14:02:45.885849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 14:02:46.033146: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-22 14:02:46.510220: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alan/.cuda/lib64:/home/alan/.cuda/extras/CUPTI/lib64:/home/alan/.cuda/lib64:/home/alan/.cuda/extras/CUPTI/lib64:
2022-11-22 14:02:46.510332: W tensorflow/stream_executor/platform/default/dso_loader.

In [3]:
PRETRAINED_MODEL_NAME = 'distilbert-base-uncased'
# PRETRAINED_MODEL_NAME = 'deepset/roberta-base-squad2'
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

## Read raw data

In [4]:
# df_train = pd.read_csv('data/Batch_answers - train_data (no-blank).csv' )

# # drop unnecessary columns
# df_train = df_train.drop(['Unnamed: 6', 'total no.: 7987'], axis=1)

# # remove quotes
# df_train['q'] = df_train['q'].str.strip('"')
# df_train['r'] = df_train['r'].str.strip('"')
# df_train["q'"] = df_train["q'"].str.strip('"')
# df_train["r'"] = df_train["r'"].str.strip('"')

# # drop duplicated rows
# df_train = df_train.drop_duplicates()

# # numerical s
# df_train['s_label'] = (df_train['s'] == 'AGREE').astype(int)

# # some information about the dataset
# print(df_train['s_label'].value_counts())
# print('# of distinct data:\t', len(df_train['id'].unique()))
# print('# of data:\t', len(df_train))

# # save cleaned dataset
# df_train.to_csv('data/train.tsv', sep='\t', index=False)

## Read DataFrame

In [5]:
df = pd.read_csv('data/train.tsv', sep='\t')
df.drop_duplicates(inplace=True)
df

,id,q,r,s,q',r',s_label
0,8,It can go both ways . We all doubt . It is wha...,True .,AGREE,It can go both ways . We all doubt . It is wha...,True .,1
1,8,It can go both ways . We all doubt . It is wha...,True .,AGREE,can go both ways . We all doubt . It is what y...,True,1
2,8,It can go both ways . We all doubt . It is wha...,True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1
3,9,"once again , you seem to support the killing o...",based on the idea that people are dispensible ...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1
4,9,"once again , you seem to support the killing o...",based on the idea that people are dispensible ...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,1
...,...,...,...,...,...,...,...
36010,10001,good thing this argument has never been done !...,"And teen sex does n't , by the very nature of ...",DISAGREE,You are much better off making theft legal and...,"And teen sex does n't , by the very nature of ...",0
36011,10002,"I know one thing , anything that happens , pol...",Was n't sinjin crowing about his plans to take...,DISAGREE,"I know one thing , anything that happens , pol...",Was n't sinjin crowing about his plans to take...,0
36012,10002,"I know one thing , anything that happens , pol...",Was n't sinjin crowing about his plans to take...,DISAGREE,FBI Arrests Three Men in Terror Plot that Targ...,Was n't sinjin crowing about his plans to take...,0
36013,10003,I enjoy Botany more than most things and I hav...,"Hi Smallax , welcome to the forum . I did a se...",AGREE,I enjoy Botany more than most things and I hav...,"Hi Smallax , welcome to the forum . I did a se...",1


In [6]:
assert np.all(df.groupby('id')['s_label'].unique().agg(len) == 1) == True, 's of an id are not all the same'
assert np.all(df.groupby('id')['q'].unique().agg(len) == 1) == True, 'q of an id are not all the same'
assert np.all(df.groupby('id')['r'].unique().agg(len) == 1) == True, 'r of an id are not all the same'

In [7]:
ids = df['id'].unique()
ss = df.groupby('id')['s'].unique().agg(lambda x: x[0])

qs = df.groupby('id')['q'].unique().agg(lambda x: x[0])
rs = df.groupby('id')['r'].unique().agg(lambda x: x[0])
q_primes  = df.groupby('id')['q\''].unique().agg(list)
r_primes  = df.groupby('id')['r\''].unique().agg(list)

assert np.all(ids == qs.index)
assert np.all(qs.index == rs.index)
assert np.all(q_primes.index == r_primes.index)
assert np.all(qs.index == q_primes.index)

print(qs)
print(rs)
print(q_primes)
print(r_primes)

id
8        It can go both ways . We all doubt . It is wha...
9        once again , you seem to support the killing o...
10       I personly would not condone an abortion , how...
11       First , there is no `` us `` on your part rega...
12                       alright how did evolution start ?
                               ...                        
9999     The ID movements form of ID states that there ...
10000    For me , it would therefore have made no diffe...
10001    good thing this argument has never been done !...
10002    I know one thing , anything that happens , pol...
10003    I enjoy Botany more than most things and I hav...
Name: q, Length: 7987, dtype: object
id
8                                                   True .
9        based on the idea that people are dispensible ...
10       This is a pretty touchy issue , and I agree wi...
11       Ah , I see . Your reasons are secret reasons ....
12                                               See above
             

### Corrpute labels

In [84]:
tdf = pd.concat([qs, q_primes, rs, r_primes, ss], axis=1)
kdf = tdf.apply(lambda row: [{'answer_start': row['q'].find(qp), 'text': qp} for qp in row['q\'']], axis=1).explode().reset_index()
kdf = pd.concat([kdf['id'], kdf.apply(lambda x: x[0]['answer_start'] == -1, axis=1)], axis=1)
bug_ids = kdf.groupby('id').any()[0]
bug_ids.sum(), bug_ids

(3780,
 id
 8        False
 9        False
 10        True
 11        True
 12        True
          ...  
 9999     False
 10000    False
 10001    False
 10002    False
 10003    False
 Name: 0, Length: 7987, dtype: bool)

In [96]:
i = 0
for idx, row in tdf[bug_ids].iterrows():
  if np.all(np.array([row['q'].find(x) for x in row['q\'']]) == -1):
    print(idx, 'q')
    print(row['q'])
    print(row['q\''])
    print()
    i += 1
    
  if np.all(np.array([row['r'].find(x) for x in row['r\'']]) == -1):
    print(idx, 'r')
    print(row['r'])
    print(row['r\''])
    print()
    i += 1
print('total', i)

681 q
I am suggesting that ( assuming a God exists ) the nature of God is unknowable in the realm in which we exist . Unlike a square or a rock or a chair or a tree which are of this physical realm , and for which we can make a definition and then test the object to determine if it fits the definition , we can not do this with God , simply because there is no way to 'test ' God within the physical realm in which we exist .
["assuming a God exists nature of God is unknowable in the realm in which we exist there is no way to 'test ' God within the physical realm", 'God the nature of God is unknowable in the realm in which we exist .']

681 r
So you would go with the unknowable argument ? Well then this is applicable to your response : So now you have said that god is undefinable or often said `` supernatural `` . T : I believe in God A : What is God ? T : I dont know . Then my question to you would be that if you dont know what God is ... then what is it that you believe in ?
['So you wo

In [8]:
df_new = pd.DataFrame({'id': ids, 'q': qs, 'r': rs, 'q\'': q_primes, 'r\'': r_primes, 's': ss}).reset_index(drop=True)
# df_new.to_pickle('data/qa_train.pkl')
df_new

,id,q,r,q',r',s
0,8,It can go both ways . We all doubt . It is wha...,True .,[It can go both ways . We all doubt . It is wh...,"[True ., True]",AGREE
1,9,"once again , you seem to support the killing o...",based on the idea that people are dispensible ...,[seem to support the killing of certain people...,[based on the idea that people are dispensible...,AGREE
2,10,"I personly would not condone an abortion , how...","This is a pretty touchy issue , and I agree wi...","[not condone an abortion , however i would n't...",[I agree with the different points of view .. ...,AGREE
3,11,"First , there is no `` us `` on your part rega...","Ah , I see . Your reasons are secret reasons ....","[no one owes you anything, No one owes you any...","[find that appalling, Your reasons are secret ...",DISAGREE
4,12,alright how did evolution start ?,See above,"[how evolution start, alright how did evolutio...","[above, See above]",DISAGREE
...,...,...,...,...,...,...
7982,9999,The ID movements form of ID states that there ...,"That , of course , is the logical fallacy know...",[Behe is happy to think that it is the christi...,[But our ignornace is not evidence of an intel...,DISAGREE
7983,10000,"For me , it would therefore have made no diffe...",It logically follows from the moral foundation...,[it would therefore have made no difference if...,[It logically follows from the moral foundatio...,AGREE
7984,10001,good thing this argument has never been done !...,"And teen sex does n't , by the very nature of ...",[good thing this argument has never been done ...,"[And teen sex does n't , by the very nature of...",DISAGREE
7985,10002,"I know one thing , anything that happens , pol...",Was n't sinjin crowing about his plans to take...,"[I know one thing , anything that happens , po...",[Was n't sinjin crowing about his plans to tak...,DISAGREE


In [53]:
qa = pipeline('question-answering', model=PRETRAINED_MODEL_NAME)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [73]:
from pprint import pprint

In [74]:
idx = 5
full_context = f'A said "{df_new["q"].iloc[idx]}", B replied "{df_new["r"].iloc[idx]}".'
question = [f'{full_context} What is the main point of A?', f'{full_context} What is the main point of B?']
context = [df_new['q'].iloc[idx], df_new['r'].iloc[idx]]
answers = [df_new['q\''].iloc[idx], df_new['r\''].iloc[idx]]
pred = qa(question=question, context=context)

pprint(question)
pprint(context)
pprint(pred)
pprint(answers)

['A said "Do you not even get statutory sick pay ?", B replied "You have to be '
 'off for more than 4 days and after that I would get the princely sum of '
 'Â£79.15 per week .. hardly worth having . Unlike teachers I can not stay at '
 'home and get paid when it snows too .". What is the main point of A?',
 'A said "Do you not even get statutory sick pay ?", B replied "You have to be '
 'off for more than 4 days and after that I would get the princely sum of '
 'Â£79.15 per week .. hardly worth having . Unlike teachers I can not stay at '
 'home and get paid when it snows too .". What is the main point of B?']
['Do you not even get statutory sick pay ?',
 'You have to be off for more than 4 days and after that I would get the '
 'princely sum of Â£79.15 per week .. hardly worth having . Unlike teachers I '
 'can not stay at home and get paid when it snows too .']
[{'answer': 'Do you not even',
  'end': 15,
  'score': 0.015520360320806503,
  'start': 0},
 {'answer': 'to be off for mor